# Data processing for FlexSUS

institution: DTU

author: [tilseb](mailto:tilseb@dtu.dk)

date created: 2020-01-30

licensed under: [GNU GENERAL PUBLIC LICENSE Version 3, 29 June 2007](https://www.gnu.org/licenses/gpl-3.0.html)

## Installation

1. Download the repository and put it in a convenient location of your choice.
1. Installation via miniconda. Get miniconda with Python 3.7 [here](https://docs.conda.io/en/latest/miniconda.html).
2. Open the anaconda promt: press `start button` , type `anaconda promt` and hit enter.
3. Navigate to the folder containing this script: `cd <path_to_folder>`
3. Use the requirements.yml file in the root directory to set up the environment: `conda env create -f requirements.yml`
5. Activate the flexus environment: `conda activate flexsus`
5. Open the notebook: `jupyter notebook`

## Description

## Content

## Script set-up

In [1]:
# import packages
import os
import pandas as pd

In [2]:
# make directories
dirs = {'output'}
for i in dirs:
    if not os.path.isdir(i):
        os.mkdir(i)

## Define output resolution

In [3]:
# set of considered countries (if empty, select all)
ccc = {'DENMARK'}

# set of considered regions (if empty, select all)
rrr = {'DK1', 'DK2'}

In [6]:
# set of years (if empty, select all)
yyy = {}  # 2025, 2035, 2045

## Load data

In [7]:
# get list of data files (w/o file extension, i.e. '.csv')
lf = [i.split('.')[0] for i in os.listdir('data')]
#lf = os.listdir('data')

## Load and process data 

In [60]:
def readData(f):
    return pd.read_csv('data/' + f + '.csv', encoding='utf8', engine='c', low_memory=False)

In [61]:
def readBigData(f):
    tmp = []
    for chunk in pd.read_csv('data/' + f + '.csv', encoding='utf8', engine='c', low_memory=False,
                             chunksize=500000):
        tmp.append(chunk)
    return pd.concat(tmp, axis=0)
    del tmp

In [9]:
def epsToZeros(df):
    df = df.replace('Eps', 0)
    return df

In [10]:
def filterYearAndCountry(df,y,c):
    if y: df = df.loc[~df.Dim3.isin(set(df.Dim3).difference(y)), :]
    if c: df = df.loc[~df.Dim4.isin(set(df.Dim4).difference(c)), :]
    return df

In [11]:
def makeValFloat(df):
    df.Val = df.Val.astype(float)
    return df

In [34]:
def groupFrame(df,i,f,aver):
    if aver:
        df = df.groupby([i,'Dim2'])['Val'].mean().unstack().T
        df.columns = pd.MultiIndex.from_product([[f + ' (mean)'], df.columns])
    else:
        df = df.groupby([i,'Dim2'])['Val'].sum().unstack().T
        df.columns = pd.MultiIndex.from_product([[f + ' (sum)'], df.columns])
    df = df.fillna(0)
    return df

In [13]:
lf

['ECONOMY_ELEC_TRANSMISSION',
 'ECONOMY_GENERATION',
 'ECONOMY_HEAT_TRANSMISSION',
 'ELEC_DEMAND',
 'ELEC_PRICE',
 'ELEC_PRICE_HOURLY',
 'ELEC_TRANSMISSION_CAPACITY',
 'ELEC_TRANSMISSION_FLOW',
 'EMISSIONS_CO2',
 'ENERGY_PRODUCTION',
 'FUEL_CONSUMPTION',
 'GENERATION_CAPACITY',
 'HEAT_DEMAND',
 'HEAT_PRICE',
 'HEAT_PRICE_HOURLY',
 'HEAT_TRANSMISSION_CAPACITY',
 'HEAT_TRANSMISSION_FLOW',
 'STORAGE_CAPACITY',
 'SYSTEM_COSTS']

### ECONOMY_ELEC_TRANSMISSION

In [14]:
lf[0]

'ECONOMY_ELEC_TRANSMISSION'

In [35]:
# ECONOMY_ELEC_TRANSMISSION
df0 = readData(lf[0])
df0 = epsToZeros(df0)
df0 = filterYearAndCountry(df0,yyy,ccc)
df0 = makeValFloat(df0)
df0 = groupFrame(df0,'Dim7',lf[0],aver=False)
df0.head(2)

ECONOMY_ELEC_TRANSMISSION (sum)                                 \
Dim7            TRANSMISSION_CAPITAL_COSTS TRANSMISSION_OPERATIONAL_COSTS   
Dim2                                                                        
Scenario1                       417.131961                       0.020614   
Scenario10                      295.729468                       0.009370   

                                                               
Dim7       TRANSMISSION_TRADE_COSTS TRANSMISSION_TRADE_INCOME  
Dim2                                                           
Scenario1                223.822992              10198.187747  
Scenario10               392.203384               5148.391004

### ECONOMY_GENERATION

In [36]:
lf[1]

'ECONOMY_GENERATION'

In [38]:
# ECONOMY_GENERATION
df1 = readData(lf[1])
df1 = epsToZeros(df1)
df1 = filterYearAndCountry(df1,yyy,ccc)
df1 = makeValFloat(df1)
df1 = groupFrame(df1,'Dim11',lf[1],aver=False)
df1.head(2)

ECONOMY_GENERATION (sum)                          \
Dim11              ELECTRICITY_SALE ENERGY_SPECIFIC_REVENUE   
Dim2                                                          
Scenario1              12210.059270            31621.444182   
Scenario10              4098.555644            29221.484573   

                                                                               \
Dim11      GENERATION_CAPITAL_COSTS GENERATION_CO2_TAX GENERATION_FIXED_COSTS   
Dim2                                                                            
Scenario1               6940.383171          86.642451            2664.404139   
Scenario10              3209.519551         219.152990            1049.003067   

                                                          \
Dim11      GENERATION_FUEL_COSTS GENERATION_GRID_TARIFFS   
Dim2                                                       
Scenario1             2620.16481               56.057248   
Scenario10             936.82982              245.660960   

                                                                  \
Dim11      GENERATION_OPERATIONAL_COSTS GENERATION_OTHER_EMI_TAX   
Dim2                                                               
Scenario1                    749.952190                 0.020144   
Scenario10                   538.911484                 0.012272   

                                                                            
Dim11      GENERATION_TAXES GENERATION_UC_COSTS    HEAT_SALE TOTAL_REVENUE  
Dim2                                                                        
Scenario1         87.905015           86.366213  5405.295318   4323.459207  
Scenario10      1971.637897          103.645321  7820.944328   3645.126609

### ECONOMY_HEAT_TRANSMISSION

In [51]:
lf[2]

'ECONOMY_HEAT_TRANSMISSION'

In [39]:
# ECONOMY_HEAT_TRANSMISSION
df2 = readData(lf[2])
df2 = epsToZeros(df2)
df2 = filterYearAndCountry(df2,yyy,ccc)
df2 = makeValFloat(df2)
df2 = groupFrame(df2,'Dim8',lf[2],aver=False)
df2.head(2)

ECONOMY_HEAT_TRANSMISSION (sum)  \
Dim8       HEAT_TRANSMISSION_CAPITAL_COSTS   
Dim2                                         
Scenario1                              0.0   
Scenario10                             0.0   

                                                                         \
Dim8       HEAT_TRANSMISSION_OPERATIONAL_COSTS TRANSMISSION_TRADE_COSTS   
Dim2                                                                      
Scenario1                             0.000225               630.736822   
Scenario10                            0.000518               461.310127   

                                      
Dim8       TRANSMISSION_TRADE_INCOME  
Dim2                                  
Scenario1                 630.736822  
Scenario10                461.310127

## ELEC_DEMAND

In [39]:
lf[3]

'ELEC_DEMAND'

In [40]:
# ELEC_DEMAND
df3 = readData(lf[3])
df3 = epsToZeros(df3)
df3 = filterYearAndCountry(df3,yyy,ccc)
df3 = makeValFloat(df3)
df3 = groupFrame(df3,'Dim6',lf[3],aver=False)
df3.head(2)

ELEC_DEMAND (sum)                                                  
Dim6           ENDO_ELBOILER   ENDO_EV ENDO_HEATPUMP ENDO_INTRASTO   EXOGENOUS
Dim2                                                                          
Scenario1           0.017731  7.389184      2.440897      0.549655  100.122479
Scenario10          0.241952  7.389184     19.760165      0.000000  100.122479

## ELEC_PRICE

In [64]:
lf[4]

'ELEC_PRICE'

In [41]:
# ELEC_PRICE
df4 = readData(lf[4])
df4 = epsToZeros(df4)
df4 = filterYearAndCountry(df4,yyy,ccc)
df4 = makeValFloat(df4)
df4 = groupFrame(df4,'Dim7',lf[4],aver=True)
df4.head(2)

,ELEC_PRICE (mean)
Dim7,Money_per_MWh
Dim2,
Scenario1,54.064789
Scenario10,58.679853


## ELEC_PRICE_HOURLY

In [70]:
lf[5]

'ELEC_PRICE_HOURLY'

In [42]:
# ELEC_PRICE_HOURLY
df5 = readData(lf[5])
df5 = epsToZeros(df5)
df5 = filterYearAndCountry(df5,yyy,ccc)
df5 = makeValFloat(df5)
df5 = groupFrame(df5,'Dim6',lf[5],aver=True)
df5.head(2)

ELEC_PRICE_HOURLY (mean)                                   \
Dim6                            S01        S11        S22        S32   
Dim2                                                                   
Scenario1                 31.226674  74.843637  46.551809  46.288509   
Scenario10                34.390692  78.171579  51.718559  54.248490   

                       
Dim6              S43  
Dim2                   
Scenario1   71.413314  
Scenario10  74.869942

## ELEC_TRANSMISSION_CAPACITY

In [78]:
lf[6]

'ELEC_TRANSMISSION_CAPACITY'

In [43]:
# ELEC_TRANSMISSION_CAPACITY
df6 = readData(lf[6])
df6 = epsToZeros(df6)
df6 = filterYearAndCountry(df6,yyy,ccc)
df6 = makeValFloat(df6)
df6['direction'] = df6.Dim5 + '_' + df6.Dim6
df6 = groupFrame(df6,'direction',lf[6],aver=False)
df6.head(2)

ELEC_TRANSMISSION_CAPACITY (sum)                              \
direction                         DK1_DE4-N DK1_DK2    DK1_NL   DK1_NO2   
Dim2                                                                      
Scenario1                         10.411282    1.77  9.375176  4.896000   
Scenario10                         9.060000    1.77  4.089523  5.124768   

                                                          
direction     DK1_SE3 DK1_UK DK2_DE4-E DK2_DK1   DK2_SE4  
Dim2                                                      
Scenario1    9.519111    4.2  5.651687     1.8  5.100000  
Scenario10  11.571565    4.2  2.955000     1.8  6.892446

## ELEC_TRANSMISSION_FLOW

In [83]:
lf[7]

'ELEC_TRANSMISSION_FLOW'

In [46]:
# ELEC_TRANSMISSION_FLOW
df7 = readData(lf[7])
df7 = epsToZeros(df7)
df7 = filterYearAndCountry(df7,yyy,{})
df7 = makeValFloat(df7)
df7['direction'] = df7.Dim5 + '_' + df7.Dim6
df7 = df7.loc[(df7.direction.str.contains('|'.join(rrr))==True), :]
df7 = groupFrame(df7,'direction',lf[7],aver=False)
df7.head(2)

ELEC_TRANSMISSION_FLOW (sum)                                  \
direction                     DE4-E_DK2  DE4-N_DK1  DK1_DE4-N   DK1_DK2   
Dim2                                                                      
Scenario1                      0.060723   0.498527  52.263303  3.300211   
Scenario10                     6.642211  11.211683  22.533587  5.471912   

                                                                            \
direction      DK1_NL   DK1_NO2    DK1_SE3     DK1_UK  DK2_DE4-E   DK2_DK1   
Dim2                                                                         
Scenario1   51.288917  7.427024  26.622734  18.645123  30.606655  1.998565   
Scenario10  15.292184  2.452366  22.249023  13.229752   8.620229  1.330232   

                                                                            
direction    DK2_SE4    NL_DK1    NO2_DK1    SE3_DK1    SE4_DK2     UK_DK1  
Dim2                                                                        
Scenario1   9.547093  4.493234  13.833772  31.765463  10.255517   6.740544  
Scenario10  0.499585  6.950268  26.196357  49.469099  28.458313  10.171865

## EMISSIONS_CO2

In [135]:
lf[8]

'EMISSIONS_CO2'

In [48]:
# EMISSIONS_CO2
df8 = readData(lf[8])
df8 = epsToZeros(df8)
df8 = filterYearAndCountry(df8,yyy,ccc)
df8 = makeValFloat(df8)
df8['technology'] = df8.Dim8 + '_' + df8.Dim9
df8 = groupFrame(df8,'technology',lf[8],aver=False)
df8.head(2)

EMISSIONS_CO2 (sum)                                             \
technology        COAL_BOILERS COAL_CHP-BACK-PRESSURE COAL_CHP-EXTRACTION   
Dim2                                                                        
Scenario1             0.000000              41.917468        6.883196e-09   
Scenario10            5.326626             894.718410        4.328434e+03   

                                                                      \
technology COAL_CONDENSING FUELOIL_BOILERS FUELOIL_CHP-BACK-PRESSURE   
Dim2                                                                   
Scenario1              0.0        0.000000                  0.000000   
Scenario10             0.0        0.439693                  1.958605   

                                                                           \
technology FUELOIL_CONDENSING LIGHTOIL_BOILERS LIGHTOIL_CHP-BACK-PRESSURE   
Dim2                                                                        
Scenario1            0.031298         0.000000                   6.170311   
Scenario10           0.031298         5.551103                   6.854302   

                                                                        \
technology LIGHTOIL_CONDENSING NATGAS_BOILERS NATGAS_CHP-BACK-PRESSURE   
Dim2                                                                     
Scenario1             9.072628       2.858547                55.022099   
Scenario10            9.072628     897.469777               211.306336   

                                                    
technology NATGAS_CHP-EXTRACTION NATGAS_CONDENSING  
Dim2                                                
Scenario1                    0.0        773.219416  
Scenario10                   0.0          6.199197

## ENERGY_PRODUCTION

In [187]:
lf[9]

'ENERGY_PRODUCTION'

In [49]:
# EMISSIONS_CO2
df9 = readData(lf[9])
df9 = epsToZeros(df9)
df9 = filterYearAndCountry(df9,yyy,ccc)
df9 = makeValFloat(df9)
df9['technology'] = df9.Dim9 + '_' + df9.Dim8 + '_' + df9.Dim10
df9 = groupFrame(df9,'technology',lf[9],aver=False)
df9.head(2)

ENERGY_PRODUCTION (sum)                                \
technology ELECTRICITY_BIOGAS_CHP-BACK-PRESSURE ELECTRICITY_BIOGAS_CONDENSING   
Dim2                                                                            
Scenario1                              0.142967                      0.000373   
Scenario10                             1.977862                      0.000250   

                                                                               \
technology ELECTRICITY_COAL_CHP-BACK-PRESSURE ELECTRICITY_COAL_CHP-EXTRACTION   
Dim2                                                                            
Scenario1                            0.046772                    6.815530e-12   
Scenario10                           0.979251                    4.641724e+00   

                                        \
technology ELECTRICITY_COAL_CONDENSING   
Dim2                                     
Scenario1                          0.0   
Scenario10                         0.0   

                                                             \
technology ELECTRICITY_ELECTRIC_INTRASEASONAL-ELECT-STORAGE   
Dim2                                                          
Scenario1                                          7.647365   
Scenario10                                         0.000000   

                                                  \
technology ELECTRICITY_FUELOIL_CHP-BACK-PRESSURE   
Dim2                                               
Scenario1                                 0.0000   
Scenario10                                0.0025   

                                           \
technology ELECTRICITY_FUELOIL_CONDENSING   
Dim2                                        
Scenario1                        0.000049   
Scenario10                       0.000049   

                                                   \
technology ELECTRICITY_LIGHTOIL_CHP-BACK-PRESSURE   
Dim2                                                
Scenario1                                0.008555   
Scenario10                               0.009506   

                                            ...                     \
technology ELECTRICITY_LIGHTOIL_CONDENSING  ... HEAT_STRAW_BOILERS   
Dim2                                        ...                      
Scenario1                         0.013303  ...           4.020378   
Scenario10                        0.013303  ...           0.147762   

                                                                \
technology HEAT_STRAW_CHP-BACK-PRESSURE HEAT_SUN_SOLAR-HEATING   
Dim2                                                             
Scenario1                     15.861445               6.048965   
Scenario10                     0.750449               1.145687   

                                                          \
technology HEAT_WASTEHEAT_BOILERS HEAT_WOODCHIPS_BOILERS   
Dim2                                                       
Scenario1                0.474354               5.069544   
Scenario10               0.474354              31.960424   

                                                                      \
technology HEAT_WOODCHIPS_CHP-BACK-PRESSURE HEAT_WOODPELLETS_BOILERS   
Dim2                                                                   
Scenario1                         83.567709                  0.00000   
Scenario10                         4.428785                  0.09084   

                                                                               \
technology HEAT_WOODPELLETS_CHP-BACK-PRESSURE HEAT_WOODPELLETS_CHP-EXTRACTION   
Dim2                                                                            
Scenario1                            2.631275                        1.066647   
Scenario10                           0.007574                        1.064476   

                                   
technology HEAT_WOODWASTE_BOILERS  
Dim2                               
Scenario1                0.831019  
Scenario10               0.799486  

[

## FUEL_CONSUMPTION

In [195]:
lf[10]

'FUEL_CONSUMPTION'

In [52]:
# FUEL_CONSUMPTION
df10 = readData(lf[10])
df10 = epsToZeros(df10)
df10 = filterYearAndCountry(df10,yyy,ccc)
df10 = makeValFloat(df10)
#df10['technology'] = df10.Dim8 + '_' + df10.Dim9
df10 = groupFrame(df10,'Dim8',lf[10],aver=False)
df10.head(2)

FUEL_CONSUMPTION (sum)                                           \
Dim8                       BIOGAS   BIOOIL       COAL   ELECTRIC   FUELOIL   
Dim2                                                                         
Scenario1                0.335388  0.00000   0.123084  10.655648  0.000117   
Scenario10               4.426154  1.38773  15.352594  20.002117  0.009120   

                                                                            \
Dim8             HEAT  LIGHTOIL  MUNIWASTE    NATGAS      STRAW        SUN   
Dim2                                                                         
Scenario1   27.200714  0.057218  32.617654  4.115172  22.098829  48.799530   
Scenario10   0.000000  0.080623  31.989164  5.520773   1.164413   2.847284   

                                                                             
Dim8       WASTEHEAT     WATER        WIND  WOODCHIPS WOODPELLETS WOODWASTE  
Dim2                                                                         
Scenario1   0.474354  0.100566  163.564812  95.554043    6.035170  0.870664  
Scenario10  0.474354  0.100566   59.727893  32.983738    2.020647  0.836717

## GENERATION_CAPACITY

In [202]:
lf[11]

'GENERATION_CAPACITY'

In [53]:
# GENERATION_CAPACITY
df11 = readData(lf[11])
df11 = epsToZeros(df11)
df11 = filterYearAndCountry(df11,yyy,ccc)
df11 = makeValFloat(df11)
df11['technology'] = df11.Dim8 + '_' + df11.Dim9
df11 = groupFrame(df11,'Dim10',lf[11],aver=False)
df11.head(2)

GENERATION_CAPACITY (sum)                                   \
Dim10                        BOILERS CHP-BACK-PRESSURE CHP-EXTRACTION   
Dim2                                                                    
Scenario1                   1.690526         23.202771       0.588961   
Scenario10                 17.240726          9.776967       2.508599   

                                                        \
Dim10      CONDENSING ELECT-TO-HEAT HYDRO-RUN-OF-RIVER   
Dim2                                                     
Scenario1   15.099901      2.344929           0.020865   
Scenario10   0.149460     14.107220           0.020865   

                                                                   \
Dim10      INTERSEASONAL-HEAT-STORAGE INTRASEASONAL-ELECT-STORAGE   
Dim2                                                                
Scenario1                    6.921221                    5.351014   
Scenario10                   0.000000                    0.000000   

                                                                           \
Dim10      INTRASEASONAL-HEAT-STORAGE SOLAR-HEATING   SOLAR-PV   WIND-OFF   
Dim2                                                                        
Scenario1                   14.620806      8.640058  41.818506  22.559131   
Scenario10                   0.000000      1.648190   1.747898   5.591300   

                       
Dim10         WIND-ON  
Dim2                   
Scenario1   24.000000  
Scenario10  13.210993

## HEAT_DEMAND

In [213]:
lf[12]

'HEAT_DEMAND'

In [54]:
# HEAT_DEMAND
df12 = readData(lf[12])
df12 = epsToZeros(df12)
df12 = filterYearAndCountry(df12,yyy,ccc)
df12 = makeValFloat(df12)
df12 = groupFrame(df12,'Dim7',lf[12],aver=False)
df12.head(2)

HEAT_DEMAND (sum)                          
Dim7           ENDO_INTERSTO ENDO_INTRASTO   EXOGENOUS
Dim2                                                  
Scenario1           4.652332      0.298073  149.765004
Scenario10          0.000000      0.000000  149.765004

## HEAT_PRICE

In [219]:
lf[13]

'HEAT_PRICE'

In [56]:
# HEAT_PRICE
df13 = readData(lf[13])
df13 = epsToZeros(df13)
df13 = filterYearAndCountry(df13,yyy,ccc)
df13 = makeValFloat(df13)
df13 = groupFrame(df13,'Dim7',lf[13],aver=True)
df13.head(2)

,HEAT_PRICE (mean)
Dim7,AVERAGE
Dim2,
Scenario1,29.245563
Scenario10,43.473431


## HEAT_PRICE_HOURLY

In [226]:
lf[14]

'HEAT_PRICE_HOURLY'

In [62]:
# HEAT_PRICE_HOURLY
df14 = readBigData(lf[14])
df14 = epsToZeros(df14)
df14 = filterYearAndCountry(df14,yyy,ccc)
df14 = makeValFloat(df14)
df14 = groupFrame(df14,'Dim7',lf[14],aver=True)
df14.head(2)

HEAT_PRICE_HOURLY (mean)                                   \
Dim7                            S01        S11        S22        S32   
Dim2                                                                   
Scenario1                 40.500100  33.761938  22.985762  22.630149   
Scenario10                54.792336  52.853598  36.305398  28.667533   

                       
Dim7              S43  
Dim2                   
Scenario1   26.349864  
Scenario10  44.748293

## HEAT_TRANSMISSION_CAPACITY

In [231]:
lf[15]

'HEAT_TRANSMISSION_CAPACITY'

In [64]:
# HEAT_TRANSMISSION_CAPACITY
df15 = readBigData(lf[15])
df15 = epsToZeros(df15)
df15 = filterYearAndCountry(df15,yyy,ccc)
df15 = makeValFloat(df15)
df15['direction'] = df15.Dim5.str.split('_').str[0] + '_' + df15.Dim6.str.split('_').str[0]
df15 = groupFrame(df15,'direction',lf[15],aver=False)
df15.head(2)

HEAT_TRANSMISSION_CAPACITY (sum)          
direction                           DK1_DK1   DK2_DK2
Dim2                                                 
Scenario1                          6001.542  6001.056
Scenario10                         6001.542  6001.056

## HEAT_TRANSMISSION_FLOW

In [243]:
lf[16]

'HEAT_TRANSMISSION_FLOW'

In [72]:
# HEAT_TRANSMISSION_FLOW
df16 = readBigData(lf[16])
df16 = epsToZeros(df16)
df16 = filterYearAndCountry(df16,yyy,ccc)
df16 = makeValFloat(df16)
df16['direction'] = df16.Dim5.str.split('_').str[0] + '-' + df16.Dim6.str.split('_').str[0]
#df16['direction'] = df16.Dim5 + '-' + df16.Dim6
#or use 'Dim6'
df16 = groupFrame(df16,'direction',lf[16],aver=False)
df16.head(2)

HEAT_TRANSMISSION_FLOW (sum)          
direction                       DK1-DK1   DK2-DK2
Dim2                                             
Scenario1                     11.834274  7.952145
Scenario10                     6.393838  4.336831

## STORAGE_CAPACITY

In [253]:
lf[17]

'STORAGE_CAPACITY'

In [73]:
# STORAGE_CAPACITY
df17 = readBigData(lf[17])
df17 = epsToZeros(df17)
df17 = filterYearAndCountry(df17,yyy,ccc)
df17 = makeValFloat(df17)
df17 = groupFrame(df17,'Dim10',lf[17],aver=False)
df17.head(2)

STORAGE_CAPACITY (sum)                              \
Dim10      INTERSEASONAL-HEAT-STORAGE INTRASEASONAL-ELECT-STORAGE   
Dim2                                                                
Scenario1                  799.197444                   21.404056   
Scenario10                   0.000000                    0.000000   

                                       
Dim10      INTRASEASONAL-HEAT-STORAGE  
Dim2                                   
Scenario1                  331.346904  
Scenario10                   0.000000

## SYSTEM_COSTS

In [262]:
lf[18]

'SYSTEM_COSTS'

In [74]:
# SYSTEM_COSTS
df18 = readBigData(lf[18])
df18 = epsToZeros(df18)
df18 = filterYearAndCountry(df18,yyy,ccc)
df18 = makeValFloat(df18)
df18 = groupFrame(df18,'Dim6',lf[18],aver=False)
df18.head(2)

SYSTEM_COSTS (sum)                                            \
Dim6       GENERATION_CAPITAL_COSTS GENERATION_CO2_TAX GENERATION_FIXED_COSTS   
Dim2                                                                            
Scenario1               6940.383171          86.642451            2664.404139   
Scenario10              3209.519551         219.152990            1049.003067   

                                                               \
Dim6       GENERATION_FUEL_COSTS GENERATION_OPERATIONAL_COSTS   
Dim2                                                            
Scenario1            2606.018031                   749.952190   
Scenario10            922.158465                   538.911484   

                                                                      \
Dim6       GENERATION_OTHER_EMI_TAX GENERATION_UC_COSTS GRID_TARIFFS   
Dim2                                                                   
Scenario1                  0.020144           86.366213  4317.979857   
Scenario10                 0.012272          103.645321  4507.583569   

                                            \
Dim6       HEAT_TRANSMISSION_CAPITAL_COSTS   
Dim2                                         
Scenario1                              0.0   
Scenario10                             0.0   

                                                              \
Dim6       HEAT_TRANSMISSION_OPERATIONAL_COSTS         TAXES   
Dim2                                                           
Scenario1                             0.000225   8188.898895   
Scenario10                            0.000518  10072.631777   

                                                                      
Dim6       TRANSMISSION_CAPITAL_COSTS TRANSMISSION_OPERATIONAL_COSTS  
Dim2                                                                  
Scenario1                  417.131961                       0.020614  
Scenario10                 295.729468                       0.009370

In [98]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18]
df_all = pd.concat(frames, axis=1, sort=False)
df_all.index.set_names([None], inplace=True)
df_all.columns.set_names([None, None], inplace=True)
df_all.to_csv('output/data_all.csv', encoding='utf8')
df_all.head(2)

ECONOMY_GENERATION (sum)                          \
                   ELECTRICITY_SALE ENERGY_SPECIFIC_REVENUE   
Scenario1              12210.059270            31621.444182   
Scenario10              4098.555644            29221.484573   

                                                                               \
           GENERATION_CAPITAL_COSTS GENERATION_CO2_TAX GENERATION_FIXED_COSTS   
Scenario1               6940.383171          86.642451            2664.404139   
Scenario10              3209.519551         219.152990            1049.003067   

                                                          \
           GENERATION_FUEL_COSTS GENERATION_GRID_TARIFFS   
Scenario1             2620.16481               56.057248   
Scenario10             936.82982              245.660960   

                                                                  \
           GENERATION_OPERATIONAL_COSTS GENERATION_OTHER_EMI_TAX   
Scenario1                    749.952190                 0.020144   
Scenario10                   538.911484                 0.012272   

                             ...    SYSTEM_COSTS (sum)  \
           GENERATION_TAXES  ... GENERATION_FUEL_COSTS   
Scenario1         87.905015  ...           2606.018031   
Scenario10      1971.637897  ...            922.158465   

                                                                  \
           GENERATION_OPERATIONAL_COSTS GENERATION_OTHER_EMI_TAX   
Scenario1                    749.952190                 0.020144   
Scenario10                   538.911484                 0.012272   

                                                                             \
           GENERATION_UC_COSTS GRID_TARIFFS HEAT_TRANSMISSION_CAPITAL_COSTS   
Scenario1            86.366213  4317.979857                             0.0   
Scenario10          103.645321  4507.583569                             0.0   

                                                              \
           HEAT_TRANSMISSION_OPERATIONAL_COSTS         TAXES   
Scenario1                             0.000225   8188.898895   
Scenario10                            0.000518  10072.631777   

                                                                      
           TRANSMISSION_CAPITAL_COSTS TRANSMISSION_OPERATIONAL_COSTS  
Scenario1                  417.131961                       0.020614  
Scenario10                 295.729468                       0.009370  

[2 rows x 176 columns]